# Prepare to run in colab

In [6]:
!git clone https://BatyshchevKirill@github.com/BatyshchevKirill/text-detoxification.git

Cloning into 'text-detoxification'...
remote: Enumerating objects: 273, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 273 (delta 111), reused 193 (delta 64), pack-reused 5
Receiving objects: 100% (273/273), 55.61 MiB | 31.38 MiB/s, done.
Resolving deltas: 100% (111/111), done.


In [10]:
import sys
import os

os.chdir('/content/text-detoxification')
BASE_PATH = "/content/text-detoxification/"
sys.path.append(BASE_PATH)

In [ ]:
!pip install -r requirements.txt

# Sample test sentences

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv(BASE_PATH + "data/interim/preprocessed_data.csv")
df_src = df['toxic'].sample(n=1000, random_state=42).values
df_tgt = df['detoxed'].sample(n=1000, random_state=42).values

np.savetxt(BASE_PATH + 'data/interim/toxic_test.txt', df_src, fmt="%s", delimiter='\n')
np.savetxt(BASE_PATH + 'data/interim/target_test.txt', df_tgt, fmt="%s", delimiter='\n')

# Run and test the baseline model

In [26]:
from src.models import baseline as b, pretrained_t5 as t5
model = b.BaselineModel("/content/text-detoxification/data/interim/toxic_words.txt")

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
from tqdm import tqdm
res = []

for text in tqdm(df_src):
    res.append(model(text))

100%|██████████| 1000/1000 [06:48<00:00,  2.45it/s]


In [29]:
with open(BASE_PATH + "data/interim/baseline_res.txt", 'w') as f:
    f.write("\n".join(res))

In [33]:
from src.models.evaluate import bleu, rouge, ToxicityClassifier

classifier = ToxicityClassifier()

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluate the toxicity of the model

In [35]:
toxicity = []
for r in tqdm(res):
    toxicity.append(float(classifier(r)))
print('Average toxicity in the results of the model: ', sum(toxicity) / len(toxicity))

100%|██████████| 1000/1000 [02:39<00:00,  6.27it/s]

Average toxicity in the results of the model:  0.3153660014080524


In [62]:
bleu_res = []
for r, tgt in zip(res, df_tgt):
    out = float(bleu(r, tgt))
    if out > 0:
        bleu_res.append(out)
print('Average bleu score of the model:', sum(bleu_res) / len(bleu_res))

Average bleu score of the model: 0.3615299800038338


In [42]:
rouge1_res = []
rouge2_res = []
for r, tgt in zip(res, df_tgt):
    res = rouge(r, tgt)
    rouge1_res.append(float(res[0]))
    rouge2_res.append(float(res[1]))
print('Average rouge1 score of the model:', sum(rouge1_res) / len(rouge1_res))
print('Average rouge2 score of the model:', sum(rouge2_res) / len(rouge2_res))

Average rouge1 score of the model: 0.5722149046957493
Average rouge2 score of the model: 0.33180146803334354


# Run and test the Pretrained T5 model

In [43]:
from src.models.pretrained_t5 import PretrainedT5

pretrained = PretrainedT5()

In [48]:
from tqdm import tqdm
res = []

for text in tqdm(df_src):
    res.append(pretrained(text))

100%|██████████| 1000/1000 [23:16<00:00,  1.40s/it]


In [49]:
with open(BASE_PATH + "data/interim/t5_res.txt", 'w') as f:
    f.write("\n".join(res))

In [50]:
toxicity = []
for r in tqdm(res):
    toxicity.append(float(classifier(r)))
print('Average toxicity in the results of the model: ', sum(toxicity) / len(toxicity))

100%|██████████| 1000/1000 [02:24<00:00,  6.90it/s]

Average toxicity in the results of the model:  0.3381002720839897


In [58]:
bleu_res = []
for r, tgt in zip(res, df_tgt):
    out = float(bleu(r, tgt))
    if out > 0:
        bleu_res.append(out)
print('Average bleu score of the model:', sum(bleu_res) / len(bleu_res))

Average bleu score of the model: 0.4175015958159734


In [59]:
rouge1_res = []
rouge2_res = []
for r, tgt in zip(res, df_tgt):
    res = rouge(r, tgt)
    rouge1_res.append(float(res[0]))
    rouge2_res.append(float(res[1]))
print('Average rouge1 score of the model:', sum(rouge1_res) / len(rouge1_res))
print('Average rouge2 score of the model:', sum(rouge2_res) / len(rouge2_res))

Average rouge1 score of the model: 0.5573721431158483
Average rouge2 score of the model: 0.30413524706289174
